## Setup and run a hyperparameter tuning job with the SageMaker SDK and HuggingFace container
1. Load data to S3 for training.
2. Configure SageMaker HuggingFace estimator.
3. Select hyperparameters to search.
4. Configure hyperparameter sweep with SageMaker Tuner.
5. Evaluate results.
6. Use best model for inference and deploy.

## pip installs
sagemaker, transformers, and datasets - versions can vary depending on needs

In [3]:
#!pip install "sagemaker>=2.48.0" "transformers==4.12.3" "datasets[s3]==1.18.3" --upgrade

imports

In [4]:
import sagemaker
import boto3
import sagemaker.huggingface
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner
)

from sagemaker.huggingface import HuggingFace

from datasets import load_dataset
from transformers import AutoTokenizer
import s3fs

from utils import summarize_hpo_results

In [5]:
session = sagemaker.Session()

role = sagemaker.get_execution_role()
role_name = role.split('/')[-1]

sagemaker_session_bucket = session.default_bucket()

In [6]:
session = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {session.default_bucket()}")
print(f"sagemaker session region: {session.boto_region_name}")

sagemaker role arn: arn:aws:iam::264639154954:role/aaca-ani-cogsci-sagemaker-studio-role
sagemaker bucket: sagemaker-us-east-1-264639154954
sagemaker session region: us-east-1


Download data from Datasets library and store in S3

In [ ]:
dataset_name = "imdb"

s3_prefix = "samples/datasets/imdb"

tokenizer_name = "distilbert-base-uncased"

In [ ]:
dataset = load_dataset(dataset_name, ignore_verifications=True)

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

#tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])
test_dataset = test_dataset.shuffle().select(range(10000))

Tokenize the data

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset  = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Convert to tensors

In [ ]:
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Upload data to S3 using the 'save_to_disk' Datasets method

In [7]:
s3 = s3fs.S3FileSystem()

training_input_path = f"s3://{session.default_bucket()}/{s3_prefix}/train"
train_dataset.save_to_disk(training_input_path, fs=s3)

test_input_path = f"s3://{session.default_bucket()}/{s3_prefix}/test"
test_dataset.save_to_disk(test_input_path, fs=s3)

print(f"train_path: {training_input_path}")
print(f"test_path: {test_input_path}")

s3://sagemaker-us-east-1-264639154954/samples/datasets/imdb/train
s3://sagemaker-us-east-1-264639154954/samples/datasets/imdb/test


## Hyperparameter tuning

Set hyperparamter values that are passed to the estimtor, but we are not seeking to tune.

In [8]:
hyperparameters = {"epochs": 1,
                   "train_batch_size": 16,
                   "model_name": "distilbert-base-uncased"
                  }

## Configure HuggingFace estimator

In [9]:
# tags very useful in multi-user accounts for tracking key information
TAGS = [{"Key": "Owner", "Value": "ccooney@aflac.com"},
        {"Key": "Environment", "Value": "Dev"}]

SageMaker has a specific estimator for use with HuggingFace (https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html)

It requires a python script to be passed (this is the main training script for your model). Other important parameters include instance type and package versioning.

In [10]:
huggingface_estimator = HuggingFace(entry_point='training_script.py',
                                    source_dir='./scripts',
                                    sagemaker_session=session,
                                    instance_type='ml.p3.2xlarge',
                                    instance_count=1,
                                    role=role,
                                    transformers_version='4.12',
                                    py_version='py38',
                                    pytorch_version='1.9',
                                    hyperparameters=hyperparameters,
                                    base_job_name='hpo-HF')

## Initialize hyperparameter ranges and the metric for evaluating model performance.

There are many hyperparameters that can be tuned the selection with largely be dependent on the type of model you are working with.

The SageMaker SDK enables setting of hyperparameter ranges through the ContinuousParameter, IntegerParameter, and CategoricalParameter methods.

Look at the arguments being parsed in training_script.py to see some other hyperparameters that could be tuned.

In [11]:
hyperparameter_ranges = {"learning_rate": ContinuousParameter(0.0001, 0.1),
                         "warmup_steps": IntegerParameter(100, 500),
                         "optimizer": CategoricalParameter(["AdamW", "Adafactor"]),
                         "weight_decay": ContinuousParameter(0.00, 0.001)}

objective_metric = "loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "loss", "Regex": "loss = ([0-9\\.]+)"}]

## Configure tuner
Pass the estimator, as well as the objective metric and the hyperparameter ranges we want to tune.
It is also possible to define the optimization strategy as 'Bayesian' | 'Random' | 'Hyperband' | 'Grid' (https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_HyperParameterTuningJobConfig.html) - Here, we go with the default Bayesian approach.

In [12]:
tuner = HyperparameterTuner(
    huggingface_estimator,
    objective_metric,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=3,
    max_parallel_jobs=1,
    objective_type=objective_type,
    tags=TAGS,
    base_tuning_job_name="hpo-HF",
)

Then call the fit method in the same way you would with a normal SageMaker estimator.

In [13]:
tuner.fit(inputs={"train": training_input_path, "test": test_input_path})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


..................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


## Evaluate the results

tuner.describe() prints details of the tuning job we created

In [16]:
tuner.describe()

{'HyperParameterTuningJobName': 'huggingface-pytorch--230119-1050',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:264639154954:hyper-parameter-tuning-job/huggingface-pytorch--230119-1050',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Minimize',
   'MetricName': 'loss'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 3,
   'MaxParallelTrainingJobs': 1},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'warmup_steps',
     'MinValue': '100',
     'MaxValue': '500',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'learning_rate',
     'MinValue': '0.0001',
     'MaxValue': '0.1',
     'ScalingType': 'Auto'}],
   'CategoricalParameterRanges': []},
  'TrainingJobEarlyStoppingType': 'Off'},
 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'loss',
   'epochs': '1',
   'model_name': '"distilbert-base-uncased"',
   'sagemaker_container_log_level': '20',
 

Print the results as a pandas DataFrame for a comparison of all hyperparameter combinations.

In [23]:
results_df = tuner.analytics().dataframe()
results_df

,learning_rate,warmup_steps,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.000767,459.0,huggingface-pytorch--230119-1050-003-5f576314,Completed,0.006932,2023-01-19 11:18:33+00:00,2023-01-19 11:27:48+00:00,555.0
1,0.018117,291.0,huggingface-pytorch--230119-1050-002-8a2cd885,Completed,0.006932,2023-01-19 11:08:17+00:00,2023-01-19 11:17:36+00:00,559.0
2,0.009935,339.0,huggingface-pytorch--230119-1050-001-26fe4220,Completed,0.006932,2023-01-19 10:52:29+00:00,2023-01-19 11:06:14+00:00,825.0


In [ ]:
results_df.sort_values('FinalObjectiveValue')

Print best results for ease

In [27]:
summarize_hpo_results(tuner.latest_tuning_job.job_name)

best score: 0.006931567098945379
best params: {'learning_rate': '0.009935247403251076', 'warmup_steps': '339'}
best job-name: huggingface-pytorch--230119-1050-001-26fe4220


{'HyperParameterTuningJobName': 'huggingface-pytorch--230119-1050',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:264639154954:hyper-parameter-tuning-job/huggingface-pytorch--230119-1050',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Minimize',
   'MetricName': 'loss'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 3,
   'MaxParallelTrainingJobs': 1},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'warmup_steps',
     'MinValue': '100',
     'MaxValue': '500',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'learning_rate',
     'MinValue': '0.0001',
     'MaxValue': '0.1',
     'ScalingType': 'Auto'}],
   'CategoricalParameterRanges': []},
  'TrainingJobEarlyStoppingType': 'Off'},
 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'loss',
   'epochs': '1',
   'model_name': '"distilbert-base-uncased"',
   'sagemaker_container_log_level': '20',
 

In [ ]:
tuner.best_training_job()

Select the best tuned model for deploying and making predictions

In [ ]:
best_estimator = tuner.best_estimator()

Deploy the model to a SageMaker Endpoint, choosing an instance type for inference

In [ ]:
predictor = huggingface_estimator.deploy(1, "ml.g4dn.xlarge")

Use your deployed model to make predictions

In [ ]:
predictor.predict({"inputs": "I can say anything bad about this movie!"})

Delete endpoint when complete

In [ ]:
predictor.delete_endpoint()